In [ ]:
!git clone https://github.com/upunaprosk/la-tda.git
%cd la-tda
!unzip data/data.zip -d ./data

In [ ]:
!pip install -r requirements.txt -q

In [ ]:
from grab_attentions import *
from features_calculation import *

# Fine-tuning a pretrained model

In [ ]:
!nvidia-smi

Sun Oct  9 19:59:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P0    43W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
epoch = 1
lr = 3e-5
decay = 1e-2
batch=32
model_save_dir = "./"
run_name = f"bert-base-cased-en-cola_{batch}_{lr}_lr_{decay}_decay_balanced"
output_dir = model_save_dir+run_name

## Training argumens

Training argumens include  


[Trainer](https://github.com/huggingface/transformers/blob/d92e22d1f28324f513f3080e5c47c071a3916721/src/transformers/training_args.py#L121) class parameters;  
Model type arguments;  

```
  --model_name_or_path MODEL_NAME_OR_PATH
                        Path to pretrained model or model identifier from
                        huggingface.co/models (default: None)
  --config_name CONFIG_NAME
                        Pretrained config name or path if not the same as
                        model_name (default: None)
  --tokenizer_name TOKENIZER_NAME
                        Pretrained tokenizer name or path if not the same as
                        model_name (default: None)
```
Data training arguments; 


```
  --task_name TASK_NAME
                        The name of the task to train on: cola, mnli, mrpc,
                        qnli, qqp, rte, sst2, stsb, wnli (default: None)
  --dataset_name DATASET_NAME
                        The name of the dataset to use (via the datasets
                        library). (default: None)
  --max_seq_length MAX_SEQ_LENGTH
                        The maximum total input sequence length after
                        tokenization. Sequences longer than this will be
                        truncated, sequences shorter will be padded. (default:
                        128)
  --train_file TRAIN_FILE
                        A csv or a json file containing the training data.
                        (default: None)
  --validation_file VALIDATION_FILE
                        A csv or a json file containing the validation data.
                        (default: None)
  --test_file TEST_FILE
                        A csv or a json file containing the test data.
                        (default: None)
  --output_dir OUTPUT_DIR
                        The output directory where the model predictions and
                        checkpoints will be written. (default: None)
  --overwrite_output_dir [OVERWRITE_OUTPUT_DIR]
                        Overwrite the content of the output directory. Use
                        this to continue training if output_dir points to a
                        checkpoint directory. (default: False)
  --do_train [DO_TRAIN]
                        Whether to run training. (default: False)
  --do_eval [DO_EVAL]   Whether to run eval on the dev set. (default: False)
  --do_predict [DO_PREDICT]
                        Whether to run predictions on the test set. (default:
                        False)
  --evaluation_strategy {no,steps,epoch}
                        The evaluation strategy to use. (default: no)
```


Balance loss function;
```
  --balance_loss        Whether to use class-balanced loss. (default: False)
```
Layers weights freezing;  


```
  --freeze              Whether to use pre-trained model without fine-tuning.
                        (default: False)
```



In [ ]:
!python train.py \
        --model_name_or_path bert-base-cased \
        --train_file data/en-cola/train.csv \
        --validation_file data/en-cola/dev.csv \
        --test_file data/en-cola/test.csv \
        --do_train \
        --do_eval \
        --do_predict\
        --num_train_epochs $epoch\
        --learning_rate $lr\
        --weight_decay $decay\
        --max_seq_length 64\
        --per_device_train_batch_size $batch\
        --output_dir $output_dir\
        --balance_loss\
        # --overwrite_output_dir

train.py:48: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  ACCURACY = load_metric("accuracy", keep_in_memory=True)
INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
grad

# Attention weights extraction

In [ ]:
# texts that extract attentions for
d_dir = f"./data/en-cola/dev.csv"

In [ ]:
!PYTHONPATH=%PYTHONPATH% python grab_attentions.py --model_dir $output_dir --data_file $d_dir

In [ ]:
# Ex. of direct function call
# grab_attention_weights_inference(output_dir, d_dir)

Refer to ./feature_src/ notebooks for feature calculation.